In [1]:
import cv2
import mediapipe as mp
import numpy as np
import json
import math

2024-09-11 17:08:46.648321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 17:08:46.684956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 17:08:46.685033: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 17:08:46.707832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 17:08:48.352874: W tensorflow/compiler/tf

In [2]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils


I0000 00:00:1726067329.890077   57402 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1726067329.895375   57472 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: AMD Radeon Vega 8 Graphics (raven, LLVM 15.0.7, DRM 3.57, 6.8.0-40-generic)


In [3]:
# Helper function to calculate angle between three points
def calculate_angle(A, B, C):
    BA = A - B
    BC = C - B
    cosine_angle = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1726067329.958662   57464 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726067330.005780   57461 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Video capture
cap = cv2.VideoCapture(0)  # Or use video file path
# For storing angles per frame
angle_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)
    
    # Check if any hand landmarks are detected
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # List of all landmark points
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            landmarks = np.array(landmarks)  # Convert to NumPy array for easier processing
            
            # Calculate angles at specific joints (example: knuckles)
            angles = {
                'thumb_base': calculate_angle(landmarks[0], landmarks[1], landmarks[2]),
                'thumb_knuckle_1': calculate_angle(landmarks[1], landmarks[2], landmarks[3]),
                'thumb_knuckle_2': calculate_angle(landmarks[2], landmarks[3], landmarks[4]),
                'index_base': calculate_angle(landmarks[0], landmarks[5], landmarks[6]),
                'index_knuckle_1': calculate_angle(landmarks[5], landmarks[6], landmarks[7]),
                'index_knuckle_2': calculate_angle(landmarks[6], landmarks[7], landmarks[8]),
                'middle_base': calculate_angle(landmarks[0], landmarks[9], landmarks[10]),
                'middle_knuckle_1': calculate_angle(landmarks[9], landmarks[10], landmarks[11]),
                'middle_knuckle_2': calculate_angle(landmarks[10], landmarks[11], landmarks[12]),
                'ring_base': calculate_angle(landmarks[0], landmarks[13], landmarks[14]),
                'ring_knuckle_1': calculate_angle(landmarks[13], landmarks[14], landmarks[15]),
                'ring_knuckle_2': calculate_angle(landmarks[14], landmarks[15], landmarks[16]),
                'pinky_base': calculate_angle(landmarks[0], landmarks[17], landmarks[18]),
                'pinky_knuckle_1': calculate_angle(landmarks[17], landmarks[18], landmarks[19]),
                'pinky_knuckle_2': calculate_angle(landmarks[18], landmarks[19], landmarks[20])

            }
            angle_data.append(angles)

            # Drawing keypoints and connections on the hand
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Displaying angles at the joints on the frame
            for joint, angle in angles.items():
                cv2.putText(frame, f"{joint}: {int(angle)} deg", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Show the frame with keypoints and angles
    cv2.imshow('Hand Tracking', frame)
    #exit video when pressing q
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    

W0000 00:00:1726067334.199524   57465 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/home/maimayeg/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [5]:
# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

# Save joint angles to a JSON file
with open('joint_angles.json', 'w') as f:
    json.dump(angle_data, f)